In [2]:
import numpy as np
import csv
import pandas as pd

### Load Data

In [3]:
data = pd.read_excel('resources/fall2017data.xlsx', delimiter=',')
data = data.fillna(0)
data
clean_data = data

### Data Processing
#### Eliminate Incomplete Entries: May have to figure out a better solution later

In [4]:
for choice in ['First', 'Second', 'Third']:
    for i in [1, 2, 3, 4, 7, 8]:
        clean_data = clean_data.iloc[np.where(clean_data['Period ' + str(i) + ' (' + choice + ' Choice)'] != 0)]
print clean_data.shape
# clean_data

(246, 24)


#### (TODO) Check that all courses requested are in the correct period

#### Make preferences table

In [5]:
# make sure each entry is unique
len(clean_data['Student Name']) == len(set(clean_data['Student Name']))
# student_ids = np.core.defchararray.add('S', np.char.mod('%d', np.arange(len(clean_data['Student Name']))))

True

In [6]:
def all_classes(clean_data):
    # returns a list of all the class names in a period
    result = []
    for period in [1, 2, 3, 4, 7, 8]:
        classes = clean_data.filter(like='Period ' + str(period))
        classes = pd.get_dummies(classes)
        result.extend(classes.columns)
        for choice in ['First', 'Second', 'Third']:
            s = 'Period ' + str(period) + ' (' + choice + ' Choice)'
            s2 = 'P' + str(period)
            result = [c.replace(s, s2) for c in result]
    return result

def preferences(clean_data):
    full_classes = all_classes(clean_data)
    for choice in ['First', 'Second', 'Third']:
        pref = clean_data.filter(like = choice + ' Choice')
        binary = pd.get_dummies(pref)
        cols = binary.columns
        # replace column names
        num_per = 7
        for i in [1, 2, 3, 4, 7, 8]:
            # add classes for each period that were not chosen
            s = 'Period ' + str(i) + ' (' + choice + ' Choice)'
            s2 = 'P' + str(i)
            cols = [c.replace(s, s2) for c in cols]
        empty = list(set(full_classes) - set(cols))
        binary.columns = cols
        for e in empty:
            binary[e] = np.zeros(len(clean_data))
        binary.to_csv('resources/' + choice + 'ChoiceBinary.csv')

preferences(clean_data)

In [7]:
first = pd.read_csv('resources/FirstChoiceBinary.csv', delimiter=',')
second = pd.read_csv('resources/SecondChoiceBinary.csv', delimiter=',')
third = pd.read_csv('resources/ThirdChoiceBinary.csv', delimiter=',')

print len(first.columns)
print len(second.columns)
print len(third.columns)

92
92
92


In [9]:
def course_caps(clean_data, class_size):
    n = 7
    period_courses = []
    for i in range(n):
        classes_in_period = clean_data.filter(like='Period ' + str(i + 1))
        classes_in_period = list(set(classes_in_period.values.flatten()))
        
        if 0 in classes_in_period:  # remove the NaNs we cast earlier
            classes_in_period.remove(0)
        
        courses = np.core.defchararray.add(np.repeat("P" + str(i + 1), len(classes_in_period)) , np.array(classes_in_period))
        period_courses.extend(courses)

    period_courses = pd.DataFrame(period_courses)

    period_courses['Size'] = np.repeat(class_size, len(period_courses))
    period_courses.to_csv('resources/CourseSize.csv')

course_caps(clean_data, 25)